In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
data = pd.read_csv("E:/Machine Learning/Random Forest/Cardekho_imputated_Regression.csv", index_col=[0])
data.head(2)

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.7,796,46.3,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.9,1197,82.0,5,550000


In [4]:
data.isna().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [5]:
data.drop(['car_name','brand'],axis=1,inplace=True)

In [6]:
data['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [7]:
num_features = [feature for feature in data.columns if data[feature].dtype != 'O']
print(len(num_features))
cat_features = [feature for feature in data.columns if data[feature].dtype == 'O']
print(len(cat_features))
discrete_features = [feature for feature in num_features if len(data[feature].unique()) <= 25]
print(len(discrete_features))
cont_features = [feature for feature in num_features if len(data[feature].unique()) > 25]
print(len(cont_features))

7
4
2
5


In [8]:
from sklearn.model_selection import train_test_split
X= data.drop('selling_price',axis=1)
Y = data['selling_price']

In [9]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
X['model'] = le.fit_transform(X['model'])

In [10]:
num_features = X.select_dtypes(exclude='object').columns
one_hot_columns = ['seller_type','fuel_type','transmission_type']

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer


numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')


preprocessor = ColumnTransformer([("OneHotEncoder", oh_transformer, one_hot_columns),
                                  ("StandardScaler", numeric_transformer, num_features) ], remainder='passthrough')


X=preprocessor.fit_transform(X)


In [11]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2, random_state=42)
X_test.shape, X_train.shape

((3083, 14), (12328, 14))

In [12]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [13]:
models = {"Adaboost": AdaBoostRegressor()}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, Y_train)

    Y_train_Pred = model.predict(X_train)
    Y_test_Pred = model.predict(X_test)

    #Training Set
    model_train_r2_score = r2_score(Y_train, Y_train_Pred)
    model_train_mae = mean_absolute_error(Y_train, Y_train_Pred)
    model_train_mse = mean_squared_error(Y_train, Y_train_Pred)
    


    #Test Set
    model_test_r2_score = r2_score(Y_test, Y_test_Pred)
    model_test_mae = mean_absolute_error(Y_test, Y_test_Pred)
    model_test_mse = mean_squared_error(Y_test, Y_test_Pred)
    
    print(list(models.keys())[i])


    print("Model Performance for Training Set")
    print("- R2 Score: {:.4f}".format(model_train_r2_score))
    print("- MAE: {:.4f}".format(model_train_mae))
    print("- MSE: {:.4f}".format(model_train_mse))
   



    print("--------------------")


    print("Model Performance for Test Set")
    print("- R2 Score: {:.4f}".format(model_test_r2_score))
    print("- MAE: {:.4f}".format(model_test_mae))
    print("- MSE: {:.4f}".format(model_test_mse))


Adaboost
Model Performance for Training Set
- R2 Score: 0.7505
- MAE: 340426.5782
- MSE: 202331309180.2152
--------------------
Model Performance for Test Set
- R2 Score: 0.6393
- MAE: 359139.6355
- MSE: 271523306519.6423


In [16]:
ada_params = {"n_estimators":[50,60,70,80],
              "loss":["linear","square","exponential"]}

In [17]:
randomcvmodels = [('KNN',AdaBoostRegressor(),ada_params)]

In [19]:
from sklearn.model_selection import RandomizedSearchCV

model_param = {}
for name, model, params in randomcvmodels:
    random = RandomizedSearchCV(estimator=model, param_distributions=params, n_iter=100, cv=3, verbose=2, n_jobs=-1)

random.fit(X_train, Y_train)

model_param['name'] = random.best_params_


for model in model_param:
    print(model_param[model])

c:\Users\IP FLEX5\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 12 is smaller than n_iter=100. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 12 candidates, totalling 36 fits
{'n_estimators': 50, 'loss': 'linear'}


In [20]:
models = {"Adaboost": AdaBoostRegressor(n_estimators=50,loss='linear')}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, Y_train)

    Y_train_Pred = model.predict(X_train)
    Y_test_Pred = model.predict(X_test)

    #Training Set
    model_train_r2_score = r2_score(Y_train, Y_train_Pred)
    model_train_mae = mean_absolute_error(Y_train, Y_train_Pred)
    model_train_mse = mean_squared_error(Y_train, Y_train_Pred)
    


    #Test Set
    model_test_r2_score = r2_score(Y_test, Y_test_Pred)
    model_test_mae = mean_absolute_error(Y_test, Y_test_Pred)
    model_test_mse = mean_squared_error(Y_test, Y_test_Pred)
    
    print(list(models.keys())[i])


    print("Model Performance for Training Set")
    print("- R2 Score: {:.4f}".format(model_train_r2_score))
    print("- MAE: {:.4f}".format(model_train_mae))
    print("- MSE: {:.4f}".format(model_train_mse))
   



    print("--------------------")


    print("Model Performance for Test Set")
    print("- R2 Score: {:.4f}".format(model_test_r2_score))
    print("- MAE: {:.4f}".format(model_test_mae))
    print("- MSE: {:.4f}".format(model_test_mse))


Adaboost
Model Performance for Training Set
- R2 Score: 0.6890
- MAE: 407294.1587
- MSE: 252271729216.3291
--------------------
Model Performance for Test Set
- R2 Score: 0.6196
- MAE: 424569.2019
- MSE: 286391111347.7377
